# Example dashboard elements for pre-night briefing

## Notebook perparation

### Load jupyter extensions

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
from tempfile import TemporaryDirectory

import numpy as np
import panel as pn
import holoviews as hv
import hvplot
import hvplot.pandas
import bokeh.models.layouts

from astropy.time import Time

import rubin_sim
from rubin_sim.scheduler.model_observatory import ModelObservatory

In [ ]:
%aimport schedview.compute.astro
%aimport schedview.collect.opsim
%aimport schedview.compute.scheduler
%aimport schedview.collect.footprint
%aimport schedview.plot.visitmap
%aimport schedview.plot.rewards
%aimport schedview.plot.visits
%aimport schedview.plot.maf

### Further preparation of the notebook

In [ ]:
pn.extension()

We need to load the `gpu.js` library used for the interactive projection explicitly.

In [ ]:
%%html
<script src="https://unpkg.com/gpu.js@latest/dist/gpu-browser.min.js"></script>

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)

## Notebook configuration

Sets the particulars for the night to be previewed.

In [ ]:
SCHEDULER_FNAME = (
    "/home/n/neilsen/devel/schedview/schedview/data/scheduler1_sample.pickle.gz"
)
OPSIM_OUTPUT_FNAME = rubin_sim.data.get_baseline()
NIGHT = Time("2023-10-04", scale="utc")
TIMEZONE = "Chile/Continental"
OBSERVATORY = ModelObservatory()
SITE = OBSERVATORY.location

## Create each element of the dashboard

### The table of astronomical events

Currently, this just returns a raw `pandas.DataFrame` with the data, which `panel` knows how to display.

If another tool is used, this may be altered to create the corresponding bokeh model explicitly.

In [ ]:
astro_events = schedview.compute.astro.night_events(NIGHT, SITE, TIMEZONE)
type(astro_events)

### Plot with the rewards for each survey

In [ ]:
(
    night_rewards_plot,
    night_rewards_data,
) = schedview.plot.rewards.create_survey_reward_plot(
    scheduler=SCHEDULER_FNAME,
    night_date=NIGHT,
    additional_visits=OPSIM_OUTPUT_FNAME,
)

In [ ]:
isinstance(night_rewards_plot, bokeh.models.layouts.LayoutDOM)

### Use the `hvplot.explorer` to show simulated visits

In [ ]:
visit_explorer, visit_explorer_data = schedview.plot.visits.create_visit_explorer(
    visits=OPSIM_OUTPUT_FNAME,
    night_date=NIGHT,
)
type(visit_explorer)

### Create interactive sky maps showing visits

In [ ]:
vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
    visits=OPSIM_OUTPUT_FNAME,
    scheduler=SCHEDULER_FNAME,
    night_date=NIGHT,
    timezone=TIMEZONE,
    observatory=OBSERVATORY,
)

In [ ]:
isinstance(vmap, bokeh.models.layouts.LayoutDOM)

### Create a plot that uses `MAF` and `matplotlib`

In [ ]:
with TemporaryDirectory() as maf_dir:
    count_ra_hist = schedview.plot.maf.create_sample_maf_metric_plot(
        OPSIM_OUTPUT_FNAME, maf_dir, NIGHT
    )

## Build a dashboard with `panel`

In [ ]:
pn.Column(
    f"<h1>Pre-night briefing for {NIGHT.iso.split()[0]}</h1>",
    pn.pane.PNG(
        "https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png", height=50
    ),
    "<h2>Astronomical Events</h2>",
    astro_events,
    "<h2>Rewards by survey, with time</h2>",
    night_rewards_plot,
    "<h2>Simulated visits</h2>",
    visit_explorer,
    "<h2>Visit map</h2>",
    vmap,
    "<h2>Count (g visits) by R.A.</h2>",
    count_ra_hist,
)